In [1]:
#!/usr/bin/env python

import argparse
import os
import os.path as osp
import torch
import models
from utils import get_log_dir, get_parameters
import dataset

configurations = {
    # same configuration as original work
    # https://github.com/shelhamer/fcn.berkeleyvision.org
    # fcn32s
    1: dict(
        max_iteration=100000,
        lr=1.0e-10,
        momentum=0.99,
        weight_decay=0.0005,
        interval_validate=4000,
    ),
    # fcn8s
#     2: dict(max_iteration=100000,
#         lr=1.0e-14,
#         momentum=0.99,
#         weight_decay=0.0005,
#         interval_validate=4000,
#         fcn16s_pretrained_model=models.FCN16s._get_pretrained(),
#     )
}

cfg = configurations[1]
out = get_log_dir('fcn32s', 1, cfg)
# resume= 'checkpoint'
resume = False
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()
torch.manual_seed(1337)
if cuda:
    torch.cuda.manual_seed(1337)

In [2]:
# 1. dataset benchmark

root = '../datasets/'
kwargs = {'num_workers': 4, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    dataset.SBDClassSeg(root, split='train', transform=True),
    batch_size=4, shuffle=True, **kwargs)

val_loader = torch.utils.data.DataLoader(
    dataset.SBDClassSeg(root, split='test', transform=True),
    batch_size=4, shuffle=True, **kwargs)

# test_loader = torch.utils.data.DataLoader(
#     dataset.VOC2012ClassSeg(
#         root, split='seg11valid', transform=True),
#     batch_size=1, shuffle=False, **kwargs)

In [3]:
# 2. model

model = models.FCN32s(n_class=21)
start_epoch = 0
start_iteration = 0
if resume:
    checkpoint = torch.load(resume)
    model.load_state_dict(checkpoint['model_state_dict'])
    start_epoch = checkpoint['epoch']
    start_iteration = checkpoint['iteration']
else:
    vgg16 = models.VGG16(pretrained=True)
    model.copy_params_from_vgg16(vgg16)
if cuda:
    model = model.cuda()


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [ ]:
# 3. optimizer
# optimizer_seg = torch.optim.Adam(seg.parameters(), lr=1e-3)
# optimizer_feat = torch.optim.Adam(res50.parameters(), lr=1e-4)

optim = torch.optim.Adam(
    [
        {'params': get_parameters(model, bias=False)},
        {'params': get_parameters(model, bias=True),
         'lr': cfg['lr'] * 2, 'weight_decay': 0},
    ],
    lr=cfg['lr'],
    momentum=cfg['momentum'],
    weight_decay=cfg['weight_decay'])

if resume:
    optim.load_state_dict(checkpoint['optim_state_dict'])

trainer = torchfcn.Trainer(
    cuda=cuda,
    model=model,
    optimizer=optim,
    train_loader=train_loader,
    val_loader=val_loader,
    out=out,
    max_iter=cfg['max_iteration'],
    interval_validate=cfg.get('interval_validate', len(train_loader)),
)
trainer.epoch = start_epoch
trainer.iteration = start_iteration
trainer.train()
